In [1]:
from FourierCKKS import *

In [2]:
import numpy as np
from fft.fft import linear_convolution_direct

# Parameters
poly_degree = 1024
data_len = poly_degree // 2
message1_length = 1500
message2_length = 1300
# For convolution, output length = L1 + L2 - 1
conv_length = message1_length + message2_length - 1

fcks = FourierCKKS(poly_degree=poly_degree)

# Generate large sample messages
orig1 = np.random.random(message1_length) + 1j * np.random.random(message1_length)
orig2 = np.random.random(message2_length) + 1j * np.random.random(message2_length)

# Encrypt with segmentation to conv_length
ct1_list = fcks.forward(orig1, target_length=conv_length)
ct2_list = fcks.forward(orig2, target_length=conv_length)

# Decrypt segments back to original lengths
rec1 = fcks.backward(ct1_list, target_length=message1_length)
rec2 = fcks.backward(ct2_list, target_length=message2_length)
success1 = np.allclose(rec1, orig1, atol=1e-1, rtol=1e-1)
success2 = np.allclose(rec2, orig2, atol=1e-1, rtol=1e-1)
print(f"Recovery successful: Message1={success1}, Message2={success2}")

# Homomorphic addition
ct_sum = fcks.cipher_add(ct1_list, ct2_list)
rec_sum = fcks.backward(ct_sum, target_length=conv_length)
true_sum = np.zeros(conv_length, dtype=complex)
true_sum[:message1_length] += orig1
true_sum[:message2_length] += orig2
add_success = np.allclose(rec_sum, true_sum, atol=1e-1, rtol=1e-1)
print(f"Addition recovery successful: {add_success}")

# Homomorphic convolution
ct_conv = fcks.cipher_conv(ct1_list, ct2_list)
rec_conv = fcks.backward(ct_conv, target_length=conv_length)
true_conv = linear_convolution_direct(orig1, orig2)
conv_success = np.allclose(rec_conv, true_conv, atol=1e-1, rtol=1e-1)
print(f"Convolution recovery successful: {conv_success}")


Recovery successful: Message1=True, Message2=True
Addition recovery successful: True
Convolution recovery successful: True
